In [1]:
import psycopg2
import glob
import pandas as pd
import os
import subprocess
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Get database credentials as dictionary
result = subprocess.run(['heroku', 'pg:credentials:url', 'DATABASE',
						 '--name', 'default', '-a', 'disch'], stdout=subprocess.PIPE, shell = True)

cred_str = result.stdout.split(b'\n')[2].decode("utf-8").strip().strip('"')
cred_str_list = cred_str.split(" ")
creds = {}
for cred in cred_str_list:
	cred_name, cred_val = cred.split('=')
	creds[cred_name] = cred_val

In [3]:
# accessing the credentials
database_name = creds['dbname']
host = creds['host']
port = creds['port']
user = creds['user']
password = creds['password']

In [9]:
# connect to database
connection = psycopg2.connect(host=host, database=database_name, user=user, password=password, port=port)
cursor = connection.cursor()

In [12]:
# from protoype question to answer selecting the top 1000 questions
PostgreSQL_select_Query = '''SELECT DISTINCT "DiscH_prototype_answer".question_id_id
                            FROM "DiscH_prototype_answer"
                            Limit 1000'''

# executing 
cursor.execute(PostgreSQL_select_Query)

data = pd.read_sql_query(PostgreSQL_select_Query, connection)

# converting the sql tables to dataframe
dataframe = pd.DataFrame(data)

In [15]:
dataframe.head(3)

,question_id_id
0,1
1,2
2,3


In [17]:
# getting the list of ids 
list_of_ids = list(dataframe.question_id_id.values)

In [21]:
# getting the information about questions in prototype question 
PostgreSQL_select_Query_2 = '''SELECT * 
                            FROM "DiscH_prototype_question"
                            '''

# executing 
cursor.execute(PostgreSQL_select_Query_2)

data2 = pd.read_sql_query(PostgreSQL_select_Query_2, connection)

# converting the sql tables to dataframe
dataframe2 = pd.DataFrame(data2)

In [22]:
dataframe2

,question_id,description,num_response,label_in_question
0,4,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,0,normal
1,5,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,0,abusive
2,6,ولعها وهيك بتكون الشقيقة قدمت اكبر خدمة لشقيقت...,0,normal
3,7,قبل ما تترحم وسام الحسن كان مشارك بقتل الحريري...,0,normal
4,8,كلنا معك معالي الوزير الملك جبران باسيل,0,normal
...,...,...,...,...
8947,8948,نفسي اكسر راسك,0,hate
8948,8949,الحياة صعبة يخوي، لازم تتعب عحالك,0,normal
8949,8950,نفسي اكسر راسك,0,hate
8950,8951,الحياة صعبة يخوي، لازم تتعب عحالك,0,normal


In [25]:
df_insertion = pd.DataFrame(columns=["question_id", "description", "num_response", "label_in_question"])
df_insertion.head(2)

,question_id,description,num_response,label_in_question


In [29]:
# retieving just the needed rows from the database
for i, id in enumerate(list_of_ids):
    tmp_data = data2[data2.question_id == id]

    description = tmp_data['description'].values[0]
    num_respones = tmp_data['num_response'].values[0]
    label_in_question = tmp_data['label_in_question'].values[0]
    
    df_insertion.loc[i] = [id, description, num_respones, label_in_question] 

In [30]:
df_insertion

,question_id,description,num_response,label_in_question
0,1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,2,normal
1,2,سورية بلد الحضارات تربطها بعلية او بحيوان,2,normal
2,3,اخي الحاج اذا شعرت انك محرجا من الانتقادات لتص...,0,normal
3,4,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,0,normal
4,5,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,0,abusive
...,...,...,...,...
995,2437,العما بقلبك قديش بتحب بلبهدلة وئام وهاب انت حق...,0,abusive
996,2439,شيل عينك عن لبنان انت و كل كلب متلك حكايتك و غ...,0,abusive
997,2440,يرثها عباد الله الصالحين,0,normal
998,2441,إذا كان حسن نصرالله شكر جبران باسيل حقيقي,0,normal


In [40]:
df_insertion['num_response'] = df_insertion['num_response'].values.astype(int)

In [41]:
df_insertion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   question_id        1000 non-null   int32 
 1   description        1000 non-null   object
 2   num_response       1000 non-null   int32 
 3   label_in_question  1000 non-null   object
dtypes: int32(2), object(2)
memory usage: 63.5+ KB


In [42]:
# insert values into database

excel_df_comments = df_insertion[["question_id", "description", "num_response", "label_in_question"]]

tpls = [tuple(x) for x in excel_df_comments.to_numpy()]

records_list_template = ','.join(['%s'] * len(tpls))
insert_query = 'insert into public."DiscH_prototype_question_select" values {}'.format(records_list_template)
cursor.execute(insert_query, tpls)
connection.commit()

In [43]:
# closing the connection
connection.close()